In [1]:
import pandas as pd
import numpy as np
import torch
from pythainlp.tokenize import word_tokenize
from keras.utils import to_categorical
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
from sklearn.metrics import accuracy_score
%matplotlib inline

Using TensorFlow backend.


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
f = open("./train.txt", "r")
x_train = np.array([word_tokenize(line.strip(),'newmm') for line in f.readlines()])
f.close()
f = open("./train_label.txt", "r")
y_train = pd.Series([line.strip() for line in f.readlines()])
f.close()
x_train.shape,y_train.shape

((24345,), (24345,))

In [4]:
x_train[0],y_train[0]

(['10',
  ' ',
  'ประโยชน์',
  'ของ',
  'ฮับ',
  'มะกอก',
  ' ',
  'เกรด',
  ' ',
  'A',
  ' ',
  'จาก',
  'ตุรกี',
  ' ',
  '1.',
  ' ',
  'ช่วย',
  'รักษา',
  'โรคเบาหวาน',
  'และ',
  'ความดัน',
  ' ',
  '2.',
  ' ',
  'ช่วย',
  'ป้องกัน',
  ' ',
  'และ',
  'รักษา',
  'โรคหัวใจ',
  ' ',
  '3.',
  ' ',
  'ช่วย',
  'ป้องกัน',
  ' ',
  'และ',
  'รักษา',
  'โรคมะเร็ง',
  ' ',
  '4.',
  ' ',
  'ช่วย',
  'บำรุง',
  'ผิวพรรณ',
  'แบบ',
  ' ',
  'Inside-Out',
  ' ',
  '5.',
  ' ',
  'ช่วย',
  'บำรุง',
  'สมอง',
  ' ',
  'และ',
  'ป้องกัน',
  'โรค',
  'อัลไซเมอร์',
  ' ',
  '6.',
  ' ',
  'ช่วย',
  'ละลาย',
  'ไขมัน',
  'ใน',
  'ร่างกาย',
  ' ',
  '7.',
  ' ',
  'ช่วย',
  'กำจัด',
  'แบคทีเรีย',
  'และ',
  'เชื้อรา',
  'ใน',
  'ร่างกาย',
  ' ',
  '8.',
  ' ',
  'ช่วย',
  'รักษาโรค',
  'ใน',
  'ท้อง',
  ' ',
  'เช่น',
  'ท้องเสีย',
  ' ',
  'หรือ',
  'อาหาร',
  'เป็นพิษ',
  ' ',
  '9.',
  ' ',
  'ช่วย',
  'บรรเทา',
  'อาการ',
  'ปวดเมื่อย',
  ' ',
  'และ',
  'รักษา',
  'โรคเหน็บชา',
  ' ',
  '

In [5]:
label=list(pd.factorize(y_train)[1])
#y_train = to_categorical(pd.factorize(y_train)[0])
print(label)
y_train=[label.index(i) for i in y_train]
print(y_train[:3])

['neu', 'neg', 'pos', 'q']
[0, 1, 0]


In [6]:
ind2word = dict()
word2ind = dict()
word2ind['UNK'] = 0
ind = 1
for ss in x_train:
    for word in ss:
        if word not in word2ind:
            ind2word[ind] = word
            word2ind[word] = ind
            ind+= 1
max_len = max([len(i) for i in x_train])
max_len

749

In [7]:
data_=[]
for x,y in zip(x_train,y_train):
    tmp =[word2ind[word] for word in x]
    if len(tmp) < max_len:
        tmp +=[0]*(max_len-len(tmp))
    data_.append([tmp,y])
data_=np.array(data_)
#np.random.shuffle(data_)
data_.shape

(24345, 2)

In [8]:
del tmp
tmp = dict({label[0]:[],label[1]:[],label[2]:[],label[3]:[]})
for i in data_:
    if i[1]==0:
        tmp[label[0]].append(i)
    elif i[1]==1:
        tmp[label[1]].append(i)
    elif i[1]==2:
        tmp[label[2]].append(i)
    elif i[1]==3:
        tmp[label[3]].append(i)
print(label[0]+':',len(tmp[label[0]]),label[1]+':',len(tmp[label[1]]),label[2]+':',len(tmp[label[2]]),label[3]+':',len(tmp[label[3]]))

neu: 14243 neg: 5713 pos: 3917 q: 472


In [9]:
data = data_

In [10]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.embeddings = nn.Embedding(len(ind2word)+1,32)
        self.conv1 = nn.Conv1d(max_len,10, 2)
        self.gru1 = nn.GRU(7,10,3)
        self.fc1 = nn.Linear(10*10 ,50)
        self.fc2 = nn.Linear(50, len(label))
        self.fc3 = nn.Softmax()

    def forward(self, x):
        # Max pooling over a (2, 2) window
        embed = self.embeddings(x)
        x = F.dropout(F.max_pool1d(F.relu(self.conv1(embed)), 4),p=0.5)
        # If the size is a square you can only specify a single number
        x,_ = self.gru1(x)
        x = x.view(-1, self.num_flat_features(x))
        x = F.dropout(F.relu(self.fc1(x)),p=0.3)
        x = F.relu(self.fc2(x))
        x = F.softmax(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
#net = net.to(device)
print(net)

Net(
  (embeddings): Embedding(31207, 32)
  (conv1): Conv1d(749, 10, kernel_size=(2,), stride=(1,))
  (gru1): GRU(7, 10, num_layers=3)
  (fc1): Linear(in_features=100, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=4, bias=True)
  (fc3): Softmax()
)


In [11]:
input= torch.tensor(list(data[:,0]))
print(input.shape)
output = net(input)
print(output)
target =torch.tensor(list(data[:,1])).long()
#target = target.view(1, -1)  # make it the same shape as output
weight=torch.tensor([1/len(tmp[label[0]]),1/len(tmp[label[1]]),1/len(tmp[label[2]]),1/len(tmp[label[3]])])
criterion = nn.CrossEntropyLoss(weight=weight)
print(target.shape,output.shape,input.shape)
loss = criterion(output, target)
print(loss,target.shape,input.shape)

torch.Size([24345, 749])
tensor([[0.2394, 0.2394, 0.2770, 0.2442],
        [0.2446, 0.2443, 0.2636, 0.2475],
        [0.2453, 0.2380, 0.2675, 0.2492],
        ...,
        [0.2499, 0.2450, 0.2601, 0.2450],
        [0.2558, 0.2422, 0.2577, 0.2443],
        [0.2590, 0.2466, 0.2536, 0.2408]], grad_fn=<SoftmaxBackward>)
torch.Size([24345]) torch.Size([24345, 4]) torch.Size([24345, 749])
tensor(1.3866, grad_fn=<NllLossBackward>) torch.Size([24345]) torch.Size([24345, 749])


/home/kok/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [12]:
net.zero_grad()     # zeroes the gradient buffers of all parameters
#out.backward(torch.randn(len(x_train), len(y_train[0])))
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
tensor([-4.0526e-06,  2.1001e-05,  2.3641e-05,  1.1044e-05, -3.6718e-05,
         1.7018e-05, -8.2706e-06,  2.1105e-06,  1.3785e-05,  3.0937e-05])


In [13]:
# create your optimizer
optimizer = optim.Adam(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

/home/kok/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [14]:
def aroundResult(out):
    y_pred=[]
    for i in output:
        mx = max(i)
        y_pred = y_pred+[list(i).index(mx)]
    #y_pred = np.array(y_pred)
    return y_pred
y_pred = aroundResult(output)

In [15]:
y_true = list(data[:len(y_pred),1])
print(loss)
print(accuracy_score(y_pred,y_true))

tensor(1.3867, grad_fn=<NllLossBackward>)
0.275374820291641


In [16]:
print(y_pred[:20],'\n',y_true[:20])

[2, 2, 2, 2, 2, 3, 2, 2, 0, 3, 3, 2, 0, 0, 2, 0, 2, 2, 2, 2] 
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 2, 3, 1, 0, 0]


In [17]:
for epoch in range(10):  # loop over the dataset multiple times
    optimizer.zero_grad()   # zero the gradient buffers
    output = net(input)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    print(loss)
    y_pred = aroundResult(output)
    y_true=list(data[:len(y_pred),1])
    print(accuracy_score(y_pred,y_true))

/home/kok/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor(1.3867, grad_fn=<NllLossBackward>)
0.475374820291641
tensor(1.3866, grad_fn=<NllLossBackward>)
0.3886218936126515
tensor(1.3864, grad_fn=<NllLossBackward>)
0.1318956664612857
tensor(1.3870, grad_fn=<NllLossBackward>)
0.049003902238652704
tensor(1.3864, grad_fn=<NllLossBackward>)
0.2831792975970425
tensor(1.3864, grad_fn=<NllLossBackward>)
0.5101663585951941
tensor(1.3864, grad_fn=<NllLossBackward>)
0.5272951324707332
tensor(1.3865, grad_fn=<NllLossBackward>)
0.5132470733210105
tensor(1.3864, grad_fn=<NllLossBackward>)
0.5247484082973917
tensor(1.3865, grad_fn=<NllLossBackward>)
0.5055658246046416


In [18]:
output = net( torch.tensor(list(data_[:,0])))
y_pred = aroundResult(output)
y_true = list(data_[:,1])
print(accuracy_score(y_pred,y_true))
print(y_pred.count(0),y_pred.count(1),y_pred.count(2),y_pred.count(3))
print(y_true.count(0),y_true.count(1),y_true.count(2),y_true.count(3))

/home/kok/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


0.5116040254672417
20517 1746 227 1855
14243 5713 3917 472


In [19]:
#rnn = nn.GRU(10, 20, 2)
#torch.randn(5, 3, 10)

In [20]:
f = open("./test.txt", "r")
x_test = np.array([word_tokenize(line.strip(),'newmm') for line in f.readlines()])
f.close()

In [21]:
test_data=[]
for x in x_test:
    tmp =[word2ind[word] if word in word2ind else word2ind['UNK'] for word in x]
    if len(tmp) < max_len:
        tmp +=[0]*(max_len-len(tmp))
    test_data.append(tmp)
output = net( torch.tensor(test_data))
y_pred=[]
for i in output:
    mx = max(i)
    y_pred.append(label[list(i).index(mx)])

/home/kok/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [22]:
y_pred[:10]

['neu', 'neu', 'neu', 'neu', 'neu', 'neu', 'neu', 'q', 'neu', 'neu']

In [23]:
import csv
with open('file.csv', mode='w') as file:
    writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['Id','Class'])
    [writer.writerow([i+1,j]) for i,j in enumerate(y_pred)]

In [24]:
read = pd.read_csv('file.csv')

In [25]:
read

,Id,Class
0,1,neu
1,2,neu
2,3,neu
3,4,neu
4,5,neu
5,6,neu
6,7,neu
7,8,q
8,9,neu
9,10,neu


In [26]:
output = Variable(torch.randn(10, 120).float())
target = Variable(torch.FloatTensor(10).uniform_(0, 120).long())
print(output)
print(target)
print(output.shape,target.shape)

tensor([[-0.0675, -0.0730,  0.5266,  ...,  0.3738, -0.2293, -0.6310],
        [ 0.7282,  0.3335,  0.6140,  ..., -0.3985, -1.2357,  2.4647],
        [ 0.8003,  1.0186, -0.3347,  ...,  1.0051, -0.0489, -0.8657],
        ...,
        [ 0.7255, -0.7106,  1.2964,  ..., -1.2586,  1.2612,  0.4618],
        [ 1.0606,  0.8302, -0.1485,  ...,  0.1834, -0.0694,  0.2561],
        [-1.2713,  0.1460, -0.6519,  ..., -1.2410,  0.9144,  0.8616]])
tensor([117,  17,  54, 107,  56,  17,  40,  94,  74,  35])
torch.Size([10, 120]) torch.Size([10])
